In [0]:
# 1.1 Data loading for classification: Using torchvision.datasets for MNIST (Done)
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

train_set = datasets.MNIST('.', download=True, train=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
dataiter = iter(train_loader)
images, labels = dataiter.next()


figure = plt.figure()
num_of_images = 10

for index in range(1, num_of_images+1):
  plt.subplot(1, 10, index)
  plt.axis('off')
  plt.imshow(images[index].numpy().squeeze(), cmap='gray')
  plt.title("{}".format(labels[index]))

In [0]:
# 1.1 Data loading for classification: Using torchvision.datasets for FashionMNIST (Done)
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

train_set = datasets.FashionMNIST('.', download=True, train=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

dataiter = iter(train_loader)
images, labels = dataiter.next()

num_of_images = 10
for index in range(1, num_of_images + 1):
    ax = plt.subplot(1, 10, index)
    ax.axis('off')
    ax.imshow(images[index,:,:,:].squeeze(), cmap='gray') # images.shape is torch.size([64,1,28,28]). squeeze() is needed for making (1,28,28) to (28,28) image.
    plt.title("{}".format(labels[index]))

In [0]:
# 1.1 Data loading for classification: Using Custom DataLoader for MNIST (Done)
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import gzip
import pickle
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor

class CustomMNIST(Dataset):
  
  def __init__(self, mnist, root_dir):
    
    with gzip.open(mnist, 'rb') as f:
      self.train_set, self.valid_set, self.test_set = pickle.load(f, encoding='bytes')
      
    self.root_dir = root_dir
    train_x, train_y = self.train_set
    self.len = train_x.shape[0]
    self.x_data = torch.from_numpy(train_x[:,:])
    self.y_data = torch.from_numpy(train_y)

  def __getitem__(self, index):
    return self.x_data[index].reshape((28,28)), self.y_data[index]
  
  def __len__(self):
    return self.len
    
url_pkl= 'https://github.com/mnielsen/neural-networks-and-deep-learning/raw/ddf26dcfa085cdccd076b7d222cce1482338eb64/data/mnist.pkl.gz'
urllib.request.urlretrieve(url_pkl, '/content/drive/My Drive/mnist.pkl.gz')
mnist = pjoin('/content/drive/My Drive/mnist.pkl.gz') # mnist pickle file path
train_set = CustomMNIST(mnist, root_dir='/content/drive/My Drive/')
train_loader = DataLoader(train_set, batch_size=10, shuffle=True, num_workers=2)

dataiter = iter(train_loader)
images, labels = dataiter.next()

for i in range(len(labels)):
  image = images[i,:,:]
  label = labels[i]

  print(i, images[i,:,:].shape, labels.shape)

  ax = plt.subplot(1, 4, i + 1)
  plt.tight_layout()
  ax.set_title('Sample #{}'.format(i))
  ax.axis('off')
  plt.imshow(images[i,:,:], cmap=cm.Greys_r)  
  plt.title("{}".format(labels[i]))
  
  if i == 3:
    plt.show()
    break

In [0]:
# 1.2 Data loading for regression: Using custom dataset for LSP (Done)
# Visualization without resizing images
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, root_dir):
    joints_dir = pjoin(data_dir, 'joints.mat')
    self.images_folder = pjoin(data_dir, 'images')
    #visualized_folder = pjoin(data_dir, 'visualized')
    mat_contents = sio.loadmat(joints_dir)
    self.joints = mat_contents['joints']
    self.images = sorted(os.listdir(self.images_folder)) # sorted list of images
      
  def __getitem__(self, index):
    im = pjoin(self.images_folder, self.images[index])
    im = plt.imread(im) # numpy array of the image
    joints = self.joints[:,:,index]
    return im, joints
  
  def __len__(self):
    self.len = print(im.shape)
    return self.len

data_dir = pjoin('/content/drive/My Drive/lsp_dataset')
data = CustomLSP(data_dir, root_dir='/content/drive/My Drive/')

################### Visualization ############################
num_of_images = 3
for index in range(0, num_of_images):
  images, joints = data[index]
  print('sample #{} has shape {}'.format(index, images.shape))
  f = plt.figure(figsize=(8,8))
  ax = f.add_subplot(1, num_of_images, index+1)
  ax.axis('off')
  ax.imshow(images)
  ax.plot([joints[0,0],joints[0,1],joints[0,2]],[joints[1,0],joints[1,1],joints[1,2]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,3],joints[0,4],joints[0,5]],[joints[1,3],joints[1,4],joints[1,5]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,6],joints[0,7],joints[0,8]],[joints[1,6],joints[1,7],joints[1,8]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,9],joints[0,10],joints[0,11]],[joints[1,9],joints[1,10],joints[1,11]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,12],joints[0,13]],[joints[1,12],joints[1,13]],marker = 'o', c='r', zorder=1)

In [0]:
# 1.2 Data loading for regression: Using custom dataset for LSP with variable batch size dataloader and splitting dataset to train and test sets (Done)
# Visualization after resizing images
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os
import math
import cv2

Batch_size = 3; # samples in one batch
train_size = 80; # 80% of the data
test_size = 100 - train_size; # 20% of the data 

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, train_size, train):
    joints_dir = pjoin(data_dir, 'joints.mat')
    images_folder = pjoin(data_dir, 'images')
    #visualized_folder = pjoin(data_dir, 'visualized')
    mat_contents = sio.loadmat(joints_dir)
    joints = mat_contents['joints']
    images = sorted(os.listdir(images_folder)) # sorted list of images
    self.train = train

    if self.train == True:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.train_image_set = []
      self.train_joint_set = []
      for i in range(0,int(self.train_len)):
        
        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        self.train_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        self.train_joint_set.append(joints_i)
        
    if self.train == False:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.test_len = (100 - train_size) * len(images) / 100
      self.test_image_set = []
      self.test_joint_set = []
      for i in range(int(self.train_len),len(images)):

        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h , w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        self.test_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        self.test_joint_set.append(joints_i)

  def __getitem__(self, index):
    if self.train == True:
      train_images = self.train_image_set[index]
      train_joints = self.train_joint_set[index]
      return train_images, train_joints
    if self.train == False:
      test_images = self.test_image_set[index]
      test_joints = self.test_joint_set[index]
      return test_images, test_joints
  
  def __len__(self):
    if self.train == True:
      return int(self.train_len)
    if self.train == False:
      return int(self.test_len)

data_dir = pjoin('/content/drive/My Drive/lsp_dataset')
train_set = CustomLSP(data_dir, train_size=80, train=True)
test_set = CustomLSP(data_dir, train_size=80, train=False)
train_loader = torch.utils.data.DataLoader(train_set, Batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, Batch_size, shuffle=True)
dataiter = iter(train_loader)
train_images, train_joints = dataiter.next()
################### Visualization ############################
num_of_images = Batch_size
for index in range(0, num_of_images):
  image = train_images[index]
  joints = train_joints[index]
  print('sample #{} has shape {}'.format(index, image.shape))
  f = plt.figure(figsize=(10,10))
  ax = f.add_subplot(1, num_of_images, index+1)
  ax.axis('off')
  ax.imshow(image)
  ax.plot([joints[0,0],joints[0,1],joints[0,2]],[joints[1,0],joints[1,1],joints[1,2]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,3],joints[0,4],joints[0,5]],[joints[1,3],joints[1,4],joints[1,5]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,6],joints[0,7],joints[0,8]],[joints[1,6],joints[1,7],joints[1,8]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,9],joints[0,10],joints[0,11]],[joints[1,9],joints[1,10],joints[1,11]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,12],joints[0,13]],[joints[1,12],joints[1,13]],marker = 'o', c='r', zorder=1)

In [0]:
# 1.3 Model training for classification with Fashion-MNIST (Done)
import torch
import torchvision
from torch import nn
from torch.nn.functional import cross_entropy
from torchvision import transforms, utils
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import progressbar
import numpy as np
from math import ceil
import pickle

class Network(nn.Module):
    def __init__(self, num_classes, input_shape):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(input_shape[0], 6, (5, 5))
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 12, (5, 5))
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.linear1 = nn.Linear(4*4*12, 120)
        self.act3 = nn.ReLU()
        self.linear2 = nn.Linear(120, 60)
        self.act4 = nn.ReLU()
        self.linear3 = nn.Linear(60,num_classes)
        #last layer should be softmax which is used for multiclass problems
        # inputs of softmax are usually normalized
        # at the end, cross entropy is used after softmax. Inputs to cross entropy are softmax output and ground-truth

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = x.view(-1, 4*4*12)
        x = self.act3(self.linear1(x))
        x = self.act4(self.linear2(x))
        x = self.linear3(x)
        return x

NUM_EPOCHS = 10
BATCH_SIZE = 10
NUM_CLASSES = 10
LR = 0.001

def main():
  # first we download the dataset here
  train_set = datasets.FashionMNIST('.', download=True, train=True, transform=transforms.ToTensor())
  test_set = datasets.FashionMNIST('.', download=True, train=False, transform=transforms.ToTensor())
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle= True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)

  # instantiate the model
  model = Network(NUM_CLASSES, (1, 28, 28))
  optimizer = torch.optim.Adam(model.parameters(), lr=LR)

  train_losses = []
  test_losses = []
  train_accuracy = [] 
  test_accuracy = [] 
  for epoch in range(NUM_EPOCHS):
    for mode, data in [("train", train_loader), ("test", test_loader)]:
  
      runningLoss = 0.
      correct = 0
      total = 0
      for step, (images, labels) in enumerate(data):   # looping over each batch data
        
        predictions = model.forward(images)
        correct += (torch.argmax(predictions, dim=-1) == labels).sum() #torch.argmax(predictions, dim=-1) find index of max values along columns in each row
        total += images.shape[0]

        loss = cross_entropy(predictions, labels)
        runningLoss += loss.item() * images.shape[0]
        
        if mode == "train":
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      (train_losses if mode == "train" else test_losses).append(runningLoss / total)
      (train_accuracy if mode == "train" else test_accuracy).append(float(correct) / total) 
      print("epochs: {} | {} | total correct: {} | total loss: {}".format(epoch,mode,correct,runningLoss/total))

  with open('/content/drive/My Drive/train_test_losses.pickle', "wb") as f: # file = 'train_test_losses.pickle'
    pickle.dump((train_losses, test_losses), f)

  with open('/content/drive/My Drive/train_test_accuracy.pickle', "wb") as g: # file = 'train_test_accuracy.pickle'
    pickle.dump((train_accuracy, test_accuracy), g)
          
if __name__ == "__main__":
    main()

######################## Plots #################################################
with open("/content/drive/My Drive/train_test_losses.pickle", "rb") as f:
        train_losses, test_losses = pickle.load(f)

with open("/content/drive/My Drive/train_test_accuracy.pickle", "rb") as g:
        train_accuracy, test_accuracy = pickle.load(g)

plt.plot(np.arange(0, len(train_losses)), train_losses, label= "train loss", color="blue")
plt.plot(np.arange(0, len(train_losses)), test_losses, label= "test loss", color="green")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

plt.plot(np.arange(0, len(train_accuracy)), train_accuracy, label= "train accuracy", color="blue")
plt.plot(np.arange(0, len(train_accuracy)), test_accuracy, label= "test accuracy", color="green")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()


In [0]:
# 1.4 Model Training for Regression: LSP (Done)
import torch
from torch import nn
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os
import math
from torch.nn.functional import cross_entropy
import pickle

class Network(nn.Module):
    def __init__(self, input_shape):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(input_shape[0], 6, (5, 5))
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 12, (5, 5))
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(12, 12, (5, 5))
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, stride=1)
        self.linear1 = nn.Linear(56*56*12, 120)
        self.act4 = nn.ReLU()
        self.linear2 = nn.Linear(120, 60)
        self.act5 = nn.ReLU()
        self.linear3 = nn.Linear(60,30) 
        self.act6 = nn.ReLU()
        self.linear4 = nn.Linear(30,28)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x.float())))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.pool3(self.act3(self.conv3(x)))
        x = x.view(-1, 56*56*12)
        x = self.act4(self.linear1(x))
        x = self.act5(self.linear2(x))
        x = self.act6(self.linear3(x))
        x = self.linear4(x)
        return x

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, train_size, train):
    joints_dir = pjoin(data_dir, 'joints.mat')
    images_folder = pjoin(data_dir, 'images')
    #visualized_folder = pjoin(data_dir, 'visualized')
    mat_contents = sio.loadmat(joints_dir)
    joints = mat_contents['joints']
    images = sorted(os.listdir(images_folder)) # sorted list of images
    self.train = train

    if self.train == True:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.train_image_set = []
      self.train_joint_set = []
      for i in range(0,int(self.train_len)):
        
        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)

        im = im.view(3,1,256*256).view(3,256,256)

        self.train_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        joints_i = torch.reshape(joints_i,(1,28)) # joints reshaped so that later joints and predictions have same shape for loss calculation
        self.train_joint_set.append(joints_i)  
        
    if self.train == False:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.test_len = (100 - train_size) * len(images) / 100
      self.test_image_set = []
      self.test_joint_set = []
      for i in range(int(self.train_len),len(images)):

        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        
        im = im.view(3,1,256*256).view(3,256,256)

        self.test_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        joints_i = torch.reshape(joints_i,(1,28)) # joints reshaped so that later joints and predictions have same shape for loss calculation
        self.test_joint_set.append(joints_i)
        
  def __getitem__(self, index):
    if self.train == True:
      train_images = self.train_image_set[index]
      train_joints = self.train_joint_set[index]
      return train_images, train_joints
    if self.train == False:
      test_images = self.test_image_set[index]
      test_joints = self.test_joint_set[index]
      return test_images, test_joints
  
  def __len__(self):
    if self.train == True:
      return int(self.train_len)
    if self.train == False:
      return int(self.test_len)

NUM_EPOCHS = 10
BATCH_SIZE = 64 # samples in one batch
LR = 0.001
train_size = 80; # 80% of the data
test_size = 100 - train_size; # 20% of the data 

def main():
  data_dir = pjoin('/content/drive/My Drive/lsp_dataset')
  train_set = CustomLSP(data_dir, train_size=80, train=True)
  test_set = CustomLSP(data_dir, train_size=80, train=False)
  train_loader = torch.utils.data.DataLoader(train_set, BATCH_SIZE, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, BATCH_SIZE, shuffle=True)

  # instantiate the model
  model = Network((3,256, 256))
  optimizer = torch.optim.Adam(model.parameters(), lr=LR)

  train_losses = []
  test_losses = []
  for epoch in range(NUM_EPOCHS):
    for mode, data in [("train", train_loader), ("test", test_loader)]:
  
      runningLoss = 0.
      correct = 0
      total = 0
      for step, (images, joints) in enumerate(data):   # looping over each batch data
        
        predictions = model.forward(images)
        total += images.shape[0]

        error = joints - predictions
        loss = error.pow(2).mean()
    
        runningLoss += loss.item() * images.shape[0]
        
        if mode == "train":
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      (train_losses if mode == "train" else test_losses).append(runningLoss / total)
      print("epochs: {} | {} | total loss: {}".format(epoch,mode,runningLoss/total))


  with open('/content/drive/My Drive/LSP_train_test_losses.pickle', "wb") as f: # file = 'LSP_train_test_losses.pickle'
    pickle.dump((train_losses, test_losses), f)
          
if __name__ == "__main__":
    main()

######################## Plots #################################################
with open("/content/drive/My Drive/LSP_train_test_losses.pickle", "rb") as f:
        train_losses, test_losses = pickle.load(f)

plt.plot(np.arange(0, len(train_losses)), train_losses, label= "train loss", color="blue")
plt.plot(np.arange(0, len(train_losses)), test_losses, label= "test loss", color="green")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

In [0]:
from google.colab import drive
drive.mount('/content/drive')